# Airbnb Feature Preprocessing
This notebook preprocesses and saves Airbnb features that are relevant to our app.

## Dataset Information
Source: insideairbnb.com/get-the-data.html

Category: New York City, New York, United States

Date Compiled: 06 August 2018

## Relevant Airbnb Features
We want preprocess and save the following list of features for use in our app's filtering functionality:
* Filter 1: Price (Filter) [not weight]- range
* Filter 2: Accommodates (filters) - Check if N<= Accommodates
* Filter 3: Number of Bedrooms (filter)- (minimum, dropdown list?)
* Filter 4: Number of Beds (filter)- input (minimum, dropdown list?)
* Filter 5: Location
    * Address Search (City, etc.)
    * Neighborhoods (dropdown)
    * Input Address → (Lat, Long), compare to Lat,Long of airbnb (tolerance 5%)
* Filter 6: Dates of Stay
    * Calendar
    * Filter: Number of Nights:   Ensure input #nights is in range of Min and Max

In [1]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.preprocessing
import re
import pickle

## Preprocess the Listings Dataset

In [2]:
listings = pd.read_csv('data/listings.csv')

listings.shape

/Users/ChentianJiang/miniconda3/envs/airbnb-rec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (43,87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(50914, 96)

In [3]:
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2515,https://www.airbnb.com/rooms/2515,20180806171147,2018-08-07,Stay at Chez Chic budget room #1,Step into our artistic spacious apartment and ...,-PLEASE BOOK DIRECTLY. NO NEED TO SEND A REQUE...,Step into our artistic spacious apartment and ...,none,NaN,...,f,NaN,NaN,f,f,strict_14_with_grace_period,f,f,3,1.42
1,2539,https://www.airbnb.com/rooms/2539,20180806171147,2018-08-07,Clean & quiet apt home by the park,Renovated apt home in elevator building.,"Spacious, renovated, and clean apt home, one b...",Renovated apt home in elevator building. Spaci...,none,Close to Prospect Park and Historic Ditmas Park,...,f,NaN,NaN,f,f,moderate,f,f,9,0.25
2,2595,https://www.airbnb.com/rooms/2595,20180806171147,2018-08-07,Skylit Midtown Castle,"Find your romantic getaway to this beautiful, ...","- Spacious (500+ft²), immaculate and nicely fu...","Find your romantic getaway to this beautiful, ...",none,Centrally located in the heart of Manhattan ju...,...,f,NaN,NaN,f,f,strict_14_with_grace_period,t,t,2,0.32
3,3330,https://www.airbnb.com/rooms/3330,20180806171147,2018-08-07,++ Brooklyn Penthouse Guestroom ++,"This is a spacious, clean, furnished master be...","Room Features: - clean, hardwood floors - 2 la...","This is a spacious, clean, furnished master be...",none,Location is GREAT!! Right off the L train in t...,...,f,NaN,NaN,f,f,strict_14_with_grace_period,f,f,3,0.32
4,3647,https://www.airbnb.com/rooms/3647,20180806171147,2018-08-07,THE VILLAGE OF HARLEM....NEW YORK !,NaN,WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...,WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...,none,NaN,...,f,NaN,NaN,f,f,strict_14_with_grace_period,t,t,1,NaN


In [4]:
# global vars

# Filter 1: Price (Filter) [not weight]- range
# Filter 2: Accommodates (filters) - Check if N<= Accommodates
# Filter 3: Number of Bedrooms (filter)- (minimum, dropdown list?)
# Filter 4: Number of Beds (filter)- input (minimum, dropdown list?)
# Filter 5: Location
# Address Search (City, etc.)
# Neighborhoods (dropdown)
# Input Address → (Lat, Long), compare to Lat,Long of airbnb (tolerance 5%)
# Filter 6: Dates of Stay
# Calendar
# Filter: Number of Nights: Ensure input #nights is in range of Min and Max

# listings['has_availability'].unique() returns 'array(['t'], dtype=object)', i.e. all listings are available
# therefore, we can take availability for granted and we don't need to include it in the column list below

KEEP_COLS = ['id',
             'price', 
             'weekly_price', 
             'monthly_price',
             'accommodates',
             'bedrooms',
             'beds',
             'city',
             'state',
             'neighbourhood',
             'latitude',
             'longitude',
             # 'has_availability',
             'minimum_nights',
             'maximum_nights'
            ]

In [5]:
listings_filters = listings[KEEP_COLS]
listings_filters.shape

(50914, 14)

In [6]:
listings_filters.head()

,id,price,weekly_price,monthly_price,accommodates,bedrooms,beds,city,state,neighbourhood,latitude,longitude,minimum_nights,maximum_nights
0,2515,$59.00,$720.00,"$1,690.00",3,1.0,2.0,New York,NY,Harlem,40.799205,-73.953676,2,21
1,2539,$149.00,$299.00,$999.00,4,1.0,3.0,Brooklyn,NY,Kensington,40.647486,-73.972370,1,730
2,2595,$225.00,"$1,995.00",NaN,2,0.0,1.0,New York,NY,Midtown,40.753621,-73.983774,1,1125
3,3330,$70.00,$650.00,"$1,900.00",2,1.0,1.0,Brooklyn,NY,Williamsburg,40.708558,-73.942362,5,730
4,3647,$150.00,NaN,NaN,2,1.0,1.0,New York,NY,Harlem,40.809018,-73.941902,3,7


In [7]:
listings_filters.columns

Index(['id', 'price', 'weekly_price', 'monthly_price', 'accommodates',
       'bedrooms', 'beds', 'city', 'state', 'neighbourhood', 'latitude',
       'longitude', 'minimum_nights', 'maximum_nights'],
      dtype='object')

In [8]:
PRICE_COLS = [col for col in listings_filters.columns if re.search('price', col)]
PRICE_COLS

['price', 'weekly_price', 'monthly_price']

In [9]:
listings_filters[PRICE_COLS].head()

,price,weekly_price,monthly_price
0,$59.00,$720.00,"$1,690.00"
1,$149.00,$299.00,$999.00
2,$225.00,"$1,995.00",NaN
3,$70.00,$650.00,"$1,900.00"
4,$150.00,NaN,NaN


In [10]:
# convert prices (object type) to numeric type

for col in PRICE_COLS:
    # get rid of commas in price
    listings_filters[col] = listings_filters[col].str.replace(',', '')

    # remove preceding dollar sign then convert to numeric type
    listings_filters[col] = pd.to_numeric(listings_filters[col].str[1:])

/Users/ChentianJiang/miniconda3/envs/airbnb-rec/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/ChentianJiang/miniconda3/envs/airbnb-rec/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
# separate out the different *types* of columns
listings_filters_type_dict = listings_filters.columns.to_series().groupby(listings_filters.dtypes).groups
listings_filters_type_dict.keys()

dict_keys([dtype('int64'), dtype('float64'), dtype('O')])

In [12]:
listings_filters_type_dict

{dtype('int64'): Index(['id', 'accommodates', 'minimum_nights', 'maximum_nights'], dtype='object'),
 dtype('float64'): Index(['price', 'weekly_price', 'monthly_price', 'bedrooms', 'beds',
        'latitude', 'longitude'],
       dtype='object'),
 dtype('O'): Index(['city', 'state', 'neighbourhood'], dtype='object')}

In [13]:
def create_data_dict(data_df, type_dict):
    int_data = data_df[type_dict[np.dtype('int')]]
    float_data = data_df[type_dict[np.dtype('float')]]
    num_data = pd.concat([int_data, float_data], axis=1)
    
    cat_data = data_df[type_dict[np.dtype('object')]]
    return {'num': num_data, 'cat':cat_data}

In [14]:
listings_filters_dict = create_data_dict(listings_filters, listings_filters_type_dict)

listings_filters_dict['num'].shape, listings_filters_dict['cat'].shape

((50914, 11), (50914, 3))

In [15]:
listings_filters_dict['num'].head()

,id,accommodates,minimum_nights,maximum_nights,price,weekly_price,monthly_price,bedrooms,beds,latitude,longitude
0,2515,3,2,21,59.0,720.0,1690.0,1.0,2.0,40.799205,-73.953676
1,2539,4,1,730,149.0,299.0,999.0,1.0,3.0,40.647486,-73.972370
2,2595,2,1,1125,225.0,1995.0,NaN,0.0,1.0,40.753621,-73.983774
3,3330,2,5,730,70.0,650.0,1900.0,1.0,1.0,40.708558,-73.942362
4,3647,2,3,7,150.0,NaN,NaN,1.0,1.0,40.809018,-73.941902


In [16]:
# strip whitespace from categorical columns and convert everything to uppercase
# to avoid duplicates as a result of spacing/case
for col in listings_filters_dict['cat'].columns:
    listings_filters_dict['cat'][col] = listings_filters_dict['cat'][col].str.strip()
    listings_filters_dict['cat'][col] = listings_filters_dict['cat'][col].str.upper()

/Users/ChentianJiang/miniconda3/envs/airbnb-rec/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/ChentianJiang/miniconda3/envs/airbnb-rec/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [17]:
listings_filters_dict['cat'].head()

,city,state,neighbourhood
0,NEW YORK,NY,HARLEM
1,BROOKLYN,NY,KENSINGTON
2,NEW YORK,NY,MIDTOWN
3,BROOKLYN,NY,WILLIAMSBURG
4,NEW YORK,NY,HARLEM


In [18]:
# put together numerical and categorical columns --> save
listings_filters_cleaned = pd.concat([listings_filters_dict['num'], listings_filters_dict['cat']], axis=1)
pickle.dump(listings_filters_cleaned, open("airbnb_listings_filters_cleaned.pickle", "wb"))

In [19]:
# one-hot encode categorical features
listings_filters_dict['cat'] = pd.get_dummies(listings_filters_dict['cat']) # this also encodes NaN/Null/NA etc. as its own category
listings_filters_dict['cat'].shape

(50914, 402)

In [20]:
listings_filters_dict['cat'].head()

,city_8425 ELMHURST AVENUE,city_ARVERNE,city_ASTORIA,city_ASTORIA - NEW YORK,city_ASTORIA NEW YORK,city_ASTORIA QUEENS,"city_ASTORIA, NEW YORK","city_ASTORIA, NYC","city_ASTORIA, QUEENS","city_ASTORIA,NEW YORK",...,neighbourhood_WESTCHESTER VILLAGE,neighbourhood_WESTERLEIGH,neighbourhood_WHITESTONE,neighbourhood_WILLIAMSBRIDGE,neighbourhood_WILLIAMSBURG,neighbourhood_WILLOWBROOK,neighbourhood_WINDSOR TERRACE,neighbourhood_WOODHAVEN,neighbourhood_WOODLAWN,neighbourhood_WOODSIDE
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# put together numerical and ENCODED categorical columns --> save
listings_filters_cleaned_encoded = pd.concat([listings_filters_dict['num'], listings_filters_dict['cat']], axis=1)
pickle.dump(listings_filters_cleaned_encoded, open("airbnb_listings_filters_cleaned_encoded.pickle", "wb"))

## Preprocess the Calendar Dataset

In [22]:
calendar = pd.read_csv('data/calendar.csv')

calendar.shape

(18582150, 4)

In [23]:
calendar.head()

,listing_id,date,available,price
0,2515,2019-08-06,t,$89.00
1,2515,2019-08-05,t,$89.00
2,2515,2019-08-04,t,$89.00
3,2515,2019-08-03,t,$89.00
4,2515,2019-08-02,t,$89.00


In [24]:
# convert dates (object type) to datetime type

calendar['date'] = pd.to_datetime(calendar['date'], format='%Y-%m-%d')
calendar['date'].dtype

dtype('<M8[ns]')

In [25]:
# convert prices (object type) to numeric type

# get rid of commas in price
calendar['price'] = calendar['price'].str.replace(',', '')

# remove preceding dollar sign then convert to numeric type
calendar['price'] = pd.to_numeric(calendar['price'].str[1:])

calendar['price'].dtype

dtype('float64')

In [26]:
# save
pickle.dump(calendar, open("airbnb_calendar_cleaned.pickle", "wb"))